In [1]:
#Import libralies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *

#You can import other libraries if you need

In [2]:
###### DO NOT MODIFY THIS PART
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
######

In [3]:
artists = pd.read_csv('./grading/artists.csv')
# Sort artists by number of paintings
artists = artists.sort_values(by=['paintings'], ascending=False)

# Create a dataframe with artists having more than 200 paintings
artists_top = artists[artists['paintings'] >= 0].reset_index()
artists_top = artists_top[['name', 'paintings']]

updated_name = "Albrecht_Dürer".replace("_", " ")
artists_top.iloc[4, 0] = updated_name
# Explore images of top artists
images_dir = './grading/images/' #Type Your Own Data Directory
artists_dirs = os.listdir(images_dir)
artists_top_name = artists_top['name'].str.replace(' ', '_').values

In [4]:
#Test set load

images_dir = './grading/images/'
artists_dirs = os.listdir(images_dir)
artists_top_name = artists_top['name'].str.replace(' ', '_').values

input_shape = (224, 224, 3,) #Same as input_shape in your train code
n_classes = artists_top.shape[0]

test_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale = 1 / 255,
                                   shear_range = 5,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_generator = test_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=input_shape[:2],
                                                    batch_size=1,
                                                    shuffle=False,
                                                    seed=seed,
                                                   )

Found 220 images belonging to 1 classes.


In [5]:
model = tf.keras.models.load_model("MyModel_ver3")
  
test_generator.reset()
output = model.predict(test_generator, steps=test_generator.n)
output = np.argmax(output, axis=-1)
prediction = pd.DataFrame(output, index=test_generator.filenames, columns=['predictions']).sort_index().to_csv('20202026_output.csv')